# Validación cruzada

## Asignación aleatoria (ejemplo con dos clases), construccion manual

In [35]:
import pandas as pd
from pandas import DataFrame
import numpy as np
from sklearn.metrics import classification_report

In [36]:
#Leer los datos, nombre de los atributos y la clase
setInicial=pd.read_csv('diabetes.csv')

In [37]:
atributosName=setInicial.columns[:-1]
atributoClase=setInicial.columns[-1]
clasesName=setInicial[setInicial.columns[-1]].drop_duplicates()
print(atributosName)
print(clasesName)

Index(['preg', 'plas', 'pres', 'skin', 'insu', 'mass', 'pedi', 'age'], dtype='object')
0    tested_positive
1    tested_negative
Name: class, dtype: object


In [38]:
#Determinar el número de pliegues
K_folds=3

#Genera una columna con un número aleatorio
setInicial['Tmp']=pd.DataFrame(np.random.rand(len(setInicial), 1))

#Se ordenan por medio del atributo aleatorio para separarlos más adelante
setInicial=setInicial.sort_values(by='Tmp')
setInicial.pop('Tmp')

#Calcular el número de instancias por pliegue
Num=int(len(setInicial)/K_folds)

In [40]:
from sklearn.neural_network import MLPClassifier

clasificador = MLPClassifier(solver='lbfgs', 
                    alpha=1e-5,
                    hidden_layer_sizes=(8), 
                    random_state=42)

#En cada iteración se genera un train y test
cont=0
f1=[]

for x in range(1,K_folds+1,1):
    print("Pliegue: ",x)

    if x==K_folds:
        Tope=len(setInicial)
    else:
        Tope=cont+Num

    #Generar el train de acuerdo a los índices de cada pliegue
    Test = setInicial.iloc[cont:Tope]
    #Generar el test, tomando como referencia los índices que no aparecen en el train
    Training = pd.concat( [setInicial.iloc[0:cont],setInicial.iloc[Tope:len(setInicial)]],axis=0)
    print("Train: ",len(Training)," intancias.\nTest: ",len(Test)," instancias.")
    cont += Num

    #A PARTIR DE AQUÍ DE INICIA CON LA SEPARACIÓN Y CLASIFICACIÓN
    valoresTrain = Training[atributosName]
    valoresTest = Test[atributosName]
    clasesTrain = Training[atributoClase]
    clasesTest = Test[atributoClase]

    ################## Modelo ###########################
    modelo = clasificador.fit(valoresTrain, clasesTrain)

    ################## Clasificar  #################
    predict = modelo.predict(valoresTest)

    ############ Evaluar ###############################
    reporte=classification_report(clasesTest, predict, labels=clasesName, output_dict=True)
    reporte2=classification_report(clasesTest, predict, labels=clasesName)#, output_dict=True)
    print(reporte2)
    f1.append(reporte['accuracy'])
    
print(max(f1),sum(f1)/len(f1),min(f1))

Pliegue:  1
Train:  512  intancias.
Test:  256  instancias.
                 precision    recall  f1-score   support

tested_positive       0.65      0.33      0.44       102
tested_negative       0.67      0.88      0.76       154

       accuracy                           0.66       256
      macro avg       0.66      0.61      0.60       256
   weighted avg       0.66      0.66      0.63       256

Pliegue:  2
Train:  512  intancias.
Test:  256  instancias.
                 precision    recall  f1-score   support

tested_positive       0.70      0.53      0.60        87
tested_negative       0.78      0.88      0.83       169

       accuracy                           0.76       256
      macro avg       0.74      0.71      0.72       256
   weighted avg       0.75      0.76      0.75       256

Pliegue:  3
Train:  512  intancias.
Test:  256  instancias.
                 precision    recall  f1-score   support

tested_positive       0.72      0.59      0.65        79
tested_negative